In [ ]:
import tensorflow as tf

In [ ]:
# a=tf.constant(5,name='input_a')
# b=tf.constant(3,name='input_a')
# c=tf.multiply(a,b,name='mul_c')
# d=tf.add(a,b,name='add_d')
# e=tf.add(c,d,name='add_e')

# sess=tf.Session()
# output=sess.run(e)

# writer=tf.summary.FileWriter('./my_graph',sess.graph)

# writer.close()
# sess.close()

In [ ]:
# Explicitly create a Graph object
graph=tf.Graph()

with graph.as_default():
    
    with tf.name_scope('variables'):
        global_step=tf.Variable(0,dtype=tf.int32,trainable=False,name='global_step')
        total_output=tf.Variable(0.0,dtype=tf.float32,trainable=False,name='total_output')
        
    # Primary transformation Operations
    with tf.name_scope('transformation'):
        
        # Separate input layer
        with tf.name_scope('input'):
            a=tf.placeholder(tf.float32,shape=[None],name='input_placeholder_a')
            
        # Separate middle layer
        with tf.name_scope('intermediate_layer'):
            b=tf.reduce_prod(a,name='product_b')
            c=tf.reduce_sum(a,name='sum_c')
            
        # Separate output layer
        with tf.name_scope('output'):
            output=tf.add(b,c,name='output')
    
    # Update Operations
    with tf.name_scope('update'):
        update_total=total_output.assign_add(output)
        increment_step=global_step.assign_add(1)
        
    # Summary Operations
    with tf.name_scope('sumaries'):
        avg=tf.div(update_total,tf.cast(increment_step,tf.float32),name='average')
        tf.summary.scalar('output_summary',output)
        tf.summary.scalar('total_summary',update_total)
        tf.summary.scalar('average_summary',avg)
        
    # Global Variables and Operations
    with tf.name_scope('global_ops'):
        init=tf.global_variables_initializer()
        merged_summaries=tf.summary.merge_all()
      
# Start a Session, using the explicitly created Graph
sess=tf.Session(graph=graph)

# Open a SummaryWriter to save summaries
writer=tf.summary.FileWriter('./improved_gragh',graph)

# Initialize Variables
sess.run(init)

def run_graph(input_tensor):
    """
    Helper function; runs the graph with given input tensor and saves summaries
    """
    feed_dict={a:input_tensor}
    _,step,summary=sess.run([output,increment_step,merged_summaries],feed_dict=feed_dict)
    writer.add_summary(summary,step)


# Run the graph with various inputs
run_graph([2,8])
run_graph([3,1,3,3])
run_graph([8])
run_graph([1,2,3])
run_graph([11,4])
run_graph([4,1])
run_graph([7,3,1])
run_graph([6,3])
run_graph([0,2])
run_graph([4,5,6])

# Write the summaries to disk
writer.flush()

writer.close()
sess.close()